In [ ]:
from bokeh.io.showing import show
from bokeh.models.callbacks import CustomJS
from RootInteractive.InteractiveDrawing.bokeh.CDSAlias import CDSAlias
from RootInteractive.InteractiveDrawing.bokeh.CustomJSNAryFunction import CustomJSNAryFunction
from RootInteractive.InteractiveDrawing.bokeh.DownsamplerCDS import DownsamplerCDS

from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import bokehDrawSA
from RootInteractive.InteractiveDrawing.bokeh.compileVarName import getOrMakeColumns

from bokeh.models.sources import ColumnDataSource
from bokeh.models.widgets import Slider
from bokeh.models.layouts import Column

from bokeh.plotting import Figure, output_file

import pandas as pd
import numpy as np
from RootInteractive.Tools.compressArray import arrayCompressionRelative16


In [ ]:
from RootInteractive.Tools.aliTreePlayer import *

In [ ]:
import ROOT

In [ ]:
f=ROOT.TFile.Open("testDownsamplingTrigger.root")
tree = f.Get("trigger")  
tree.Show(0)
tree.SetAlias("weight1","((triggerMask&0x1)>0)*max(weight,0.01)")

In [ ]:
df=tree2Panda(tree, [".*"], "", exclude=["C"], columnMask=[["B", "b"]])

In [ ]:
df["isPion"]=df["pdgCode"]==211
df["isKaon"]=df["pdgCode"]==321
df["isProton"]=df["pdgCode"]==2212
df["isLambda"]=df["pdgCode"]==3122
#
df["isPionW1"]=(df["pdgCode"]==211)*df["weight1"]
df["isKaonW1"]=(df["pdgCode"]==321)*df["weight1"]
df["isProtonW1"]=(df["pdgCode"]==2212)*df["weight1"]
df["isLambdaW1"]=(df["pdgCode"]==3122)*df["weight1"]
#df["weight1"]=df.eval("((triggerMask&0x1)>0)*max(weight,0.01)")

In [ ]:
df["isPionW1"]=df.eval("weight1*(pdgCode==211)")
df["isKaonW1"]=df.eval("weight1*(pdgCode==321)")
df["isProtonW1"]=df.eval("weight1*(pdgCode==2212)")
df["isLambdaW1"]=df.eval("weight1*(pdgCode==3122)")

In [ ]:
parameterArray = [
    {"name": "size", "value":7, "range":[0, 30]},
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px"]},
    {"name": "paramX", "value":10, "range": [-20, 20]},
    {"name": "C_cut", "value": 1, "range": [0, 1]}
]

In [ ]:
aliasArray = [
    {"name": "ratio_PionAll", "variables": ["entries", "entriesPion"], "func": "return entriesPion / entries","context": "histoPt"},
    {"name": "ratio_KaonAll", "variables": ["entries", "entriesKaon"], "func": "return entriesKaon / entries","context": "histoPt"},
    {"name": "ratio_ProtonAll", "variables": ["entries", "entriesProton"], "func": "return entriesProton / entries","context": "histoPt"},
    {"name": "ratio_LambdaAll", "variables": ["entries", "entriesLambda"], "func": "return entriesLambda / entries","context": "histoPt"},
    #
    {"name": "entriesPionErr", "variables": ["entriesPion"], "func": "return Math.sqrt(entriesPion)","context": "histoPt"},
    {"name": "entriesKaonErr", "variables": ["entriesKaon"], "func": "return Math.sqrt(entriesKaon)","context": "histoPt"},
    {"name": "entriesProtonErr", "variables": ["entriesProton"], "func": "return Math.sqrt(entriesProton)","context": "histoPt"},
    {"name": "entriesLambdaErr", "variables": ["entriesLambda"], "func": "return Math.sqrt(entriesLambda)","context": "histoPt"},
    #
    {"name": "ratio_PionAll", "variables": ["entries", "entriesPion"], "func": "return entriesPion / entries","context": "histoPtWeighted"},
    {"name": "ratio_KaonAll", "variables": ["entries", "entriesKaon"], "func": "return entriesKaon / entries","context": "histoPtWeighted"},
    {"name": "ratio_ProtonAll", "variables": ["entries", "entriesProton"], "func": "return entriesProton / entries","context": "histoPtWeighted"},
    {"name": "ratio_LambdaAll", "variables": ["entries", "entriesLambda"], "func": "return entriesLambda / entries","context": "histoPtWeighted"},
    #
    {"name": "entriesPionErr", "variables": ["entriesPion"], "func": "return Math.sqrt(entriesPion*100)","context": "histoPtWeighted"},
    {"name": "entriesKaonErr", "variables": ["entriesKaon"], "func": "return Math.sqrt(entriesKaon*100)","context": "histoPtWeighted"},
    {"name": "entriesProtonErr", "variables": ["entriesProton"], "func": "return Math.sqrt(entriesProton*100)","context": "histoPtWeighted"},
    {"name": "entriesLambdaErr", "variables": ["entriesLambda"], "func": "return Math.sqrt(entriesLambda*100)","context": "histoPtWeighted"},
]

In [ ]:
histoArray = [
     { "name": "histoPt", "variables": ["pt"], "nbins": 100, "histograms": {
            "entries": None,
            "entriesPion": {"weights": "isPion"},
            "entriesKaon": {"weights": "isKaon"},
            "entriesProton": {"weights": "isProton"},
            "entriesLambda": {"weights": "isLambda"},
        }
    },
    { "name": "histoPtWeighted", "variables": ["pt"], "nbins": 100, "histograms": {
            "entries": {"weights": "weight1"},
            "entriesPion": {"weights": "isPionW1"},
            "entriesKaon": {"weights": "isKaonW1"},
            "entriesProton": {"weights": "isProtonW1"},
            "entriesLambda": {"weights": "isLambdaW1"},
        }
    }
]


In [ ]:

figureArray = [
    [['bin_center'], ['entriesPion', 'entriesKaon',"entriesProton","entriesLambda"], 
                     {"source":"histoPt","errY":["entriesPionErr","entriesKaonErr","entriesProtonErr","entriesLambdaErr"] }],
    [['bin_center'], ['ratio_PionAll',"ratio_KaonAll","ratio_ProtonAll","ratio_LambdaAll"], {"source":"histoPt"}],
    [['bin_center'], ['entriesPion', 'entriesKaon',"entriesProton","entriesLambda"], 
                     {"source":"histoPtWeighted","errY":["entriesPionErr","entriesKaonErr","entriesProtonErr","entriesLambdaErr"] }],
    [['bin_center'], ['ratio_PionAll',"ratio_KaonAll","ratio_ProtonAll","ratio_LambdaAll"], {"source":"histoPtWeighted"}],
    {"size":"size"}
]

widgetParams=[
    ['range', ['pt']],
    ['multiSelect',["pdgCode"]],
    ['multiSelect',["triggerMask"]],
    #['slider','F', ['@min()','@max()','@med','@min()','@median()+3*#tlm()']], # to be implmneted
    ['slider',["size"]],
    ['select',["legendFontSize"]],
    ['slider',["C_cut"]],
    ['slider',["paramX"]],
]

widgetLayoutDesc={
    "Selection": [[0, 1,2 ], {'sizing_mode': 'scale_width'}],
    "Graphics": [[3,4,5,6], {'sizing_mode': 'scale_width'}],
#    "CustomJS functions": [[7, 8]]
    }

figureLayoutDesc=[
    [0, 1, {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 200}],
    [2, 3, {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 200}],
]
tooltips = [("pt", "(@pt)"),]

#bokehDrawSA.fromArray(df, None, figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips, parameterArray=parameterArray,
#                          widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", nPointRender=300, jsFunctionArray=jsFunctionArray,
#                           aliasArray=aliasArray, histogramArray=histoArray)
output_file("downsamplingTrigger.html")

bokehDrawSA.fromArray(df,"pt<5", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips, parameterArray=parameterArray,
                          widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", nPointRender=300, histogramArray=histoArray, 
                      arrayCompression=arrayCompressionRelative16, aliasArray=aliasArray)

# backup

In [ ]:
c=ROOT.TCanvas()
tree.Draw("weight1","triggerMask==1")
c.Draw()